# Домашнее задание 1 - twitter

# Задание 1

Прочтём файл с данными в список tweets - это список словарей. Узнаем, сколько
всего окажется твитов, посмотрев длину списка. (Их **2556**).

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on


import json
import re
import string
from pprint import pprint
from collections import Counter


file = 'data/hw_3_twitter.json'

tweets = list()
for line in open(file, 'r', encoding='utf-8'):
    tweets.append(json.loads(line))

total_num_of_tweets = len(tweets)

**Ответ на задание 1:**

In [2]:
print(total_num_of_tweets)

2556


### Задание 2

Информация об удалённых твитах устроена особым образом: при конвертации из json в объект питона это будет словарь с одним ключом - *delete* - это мы и можем проверить, используя метод *.keys()* словарей. В переменную *num_of_deleted_tweets* мы и записали количество удалённых твитов (**362**), которое потом разделим на общее число твитов, чтобы получить ответ.

In [3]:
num_of_deleted_tweets = 0
for tweet in tweets:
    if list(tweet.keys())[0] == 'delete':
        num_of_deleted_tweets += 1

**Ответ на задание 2:** (округлен самим питоном)

In [4]:
print('Процент удалённых твитов: {:%}'.format(
    num_of_deleted_tweets / total_num_of_tweets))

Процент удалённых твитов: 14.162754%


# Задание 3



Для каждого твита, если есть ключ *lang*, то в каунтер по ключу, равному значению `tweeet['lang']` прибавляем единицу.

In [5]:
popular_langs_of_tweets = Counter()
for tweet in tweets:
    if 'lang' in tweet.keys():
        popular_langs_of_tweets[tweet['lang']] += 1

**Ответ на задание 3:** языки по убыванию по популярности с количеством твитов на них (20 штук, как и советовали в tg)

In [6]:
for lang, num in popular_langs_of_tweets.most_common(20):
    if lang != 'und':
        print('{}\t{}'.format(lang, num))

en	719
ja	438
es	173
ko	149
th	123
ar	119
in	71
pt	69
fr	35
tr	30
tl	29
hi	23
ru	15
fa	8
zh	8
ca	7
ur	6
nl	6
de	6


Твиты с неизвестным языком не учитываются. Их *117*

In [7]:
print(popular_langs_of_tweets['und'])

117


# Задание 4



Чтобы найти число пользователей, которые сделали 2+ твита, создадим список пользователей *users* и проходя список твитов будем в *users* вносить маленький список из двух элементов: уникального id и количество твитов человека с этим id. Когда мы встречаем user'а в первый раз, внесён будет список `[id, 1]`. Поскольку это список списков и все вложеные списки имеют одинаковую длину, то проверять существование id удобно применяя функцию `zip()` к `*users` - отдельным маленьким спискам. Такое применение даёт два списка одинаковой длины, в первом из которых - все id на текущий момент, во втором - количества твитов. id и количества твитов находятся в таких списках на одинаковых местах, что позволяет, (если такой id уже есть), увеличить на 1 количество твитов во вложенном списке исходного списка *users*, используя индекс id в разбитом списке.

В самом начале, условием `if list(tweet.keys())[0] != 'delete'` конечно проверяется, чтобы это не был удалённый твит.

In [8]:
users = list()

for tweet in tweets:
    if list(tweet.keys())[0] != 'delete':
        id = tweet['user']['id']
        if users:
            ids = list(zip(*users))[0]
            if id in ids:
                users[ids.index(id)][1] += 1
            else:
                users.append([id, 1])
        else:
            users.append([id, 1])

Ответ вычисляется просто: из общего количества пользователей нужно вычесть количество пользователей, у которых ровно один твит (0 твитов и меньше быть не может, мы вносили так, чтобы их было 1+), что также делается с помощью `zip(*users)`, но на этот раз нас интересует второй разделённый список - список количеств.

Итак, **ответ на задание 4:** да, такие пользователи есть, их 25

In [9]:
more_than_one_tweet = len(users) - list(zip(*users))[1].count(1)
print(more_than_one_tweet)

25


# Задание 5

Для каждого твита (если он не удалённый), если список *hashtags* не пуст, то обойдем все словари в нём (каждый словарь описывает один хэштег), и достанем из каждого словаря текст хэштега. Для начала внесём эти хэштеги в список. Позже к нему мы применим Counter. 

In [10]:
hashtags_list = list()

for tweet in tweets:
    if list(tweet.keys())[0] != 'delete':
        hashtags = tweet['entities']['hashtags']
        if hashtags:
            for hashtag in hashtags:
                hashtags_list.append(hashtag['text'])

**Ответ на задание 5:** *не выравнивается, извините(*

In [11]:
print('{:<30s}{:>5s}'.format('Hashtag', 'Count'))
for hashtag, count in Counter(hashtags_list).most_common(20):
    print('#{:<30s}{:5>d}'.format(hashtag, count))

Hashtag                       Count
#BTS                           17
#방탄소년단                         13
#AMAs                          11
#人気投票ガチャ                       8
#태형                            7
#뷔                             6
#BTSinChicago                  5
#BTSLoveYourselfTour           5
#오늘의방탄                         5
#PledgeForSwachhBharat         5
#MPN                           5
#PCAs                          4
#V                             4
#시카고1회차공연                      4
#เป๊กผลิตโชค                   4
#JIMIN                         4
#running                       3
#NCT                           3
#지민                            3
#WajahmuPlastik                3


# Задание 6

**Важно для номеров 5 и 6** В телеграме я описывал проблему, когда твиты не имеющие ни retweet_status, ни quoted_status_id тем не менее похожи на цитаты, тем что имеют внутри структуру **`extended_tweet{}`**, которая характерна лишь для цитат согласно [twitter API docs (ближе к концу страницы, если якорь в ссылке не работает)](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object#retweets-quotes). Самый первый [твит](https://twitter.com/i/web/status/1047350533437239297) удовлетворяющий условиям внутри цикла ровно таков. Для таких твитов я посмотрел внутрь *extended_tweet* и взял текст оттуда. Однако там кроме полного текста, есть ещё и хэштеги, которые не уместились во внешнее `entities{}`, что могло быть важно для предыдущего задания. К сожалению, я не очень успеваю поправить номер 5, и оставил его как было, не учитывая описанные факты.

Собственно, описание кода: для каждого твита проверим, что он не удалён `(list(tweet.keys())[0] != 'delete'` не ретвит `not 'retweeted_status' in tweet` и не цитата `not 'quoted_status_id' in tweet`. Если всё в порядке, проверим, чтобы язык был английским. После возьмём текст твита (двумя разными способами, в зависимости от того, есть ключ *extended_tweet* в словаре *tweet* или нет), и добавим в *words* (Counter) результат применения чистящей функции к тексту.

Функция очистки довольно простая, для ссылок несложная регулярка, для эмоджи удаление определённых байтовых последовательностей *(или это кодпоинты? не очень разобрался)*, пришлось найти в интернете. Пунктуация удаляется проверкой каждого символа и созданием новой строки из непунктуационных символов. Упоминания (mentions) не удалял

In [12]:
def clear_string(line):

    # удалить ссылки
    url_reg = re.compile('http[s]?:\/\/\S+.\S+(?:\/|\/\S+)*')
    line = url_reg.sub('', line)

    # удалить упоминания
    line = re.sub(r'@\S+', '', line)

    # удалить эмодзи
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    line = RE_EMOJI.sub(r'', line)

    # удалить пунктуацию, разделить на слова и сделать строчными буквы
    res_line = ''
    for c in line:
        if c not in string.punctuation and ord(c) < 128:
            res_line += c
    return [word.lower() for word in res_line.split()]


words = Counter()
for tweet in tweets:
    if (list(tweet.keys())[0] != 'delete' and 'retweeted_status' not in tweet and
            'quoted_status_id' not in tweet):
        lang = tweet.get('lang')
        if lang == 'en':
            if 'extended_tweet' in tweet:
                words_to_add = clear_string(
                    tweet['extended_tweet']['full_text'])
                words.update(words_to_add)

            else:
                words_to_add = clear_string(tweet['text'])
                words.update(words_to_add)

4:36: W605 invalid escape sequence '\/'
4:38: W605 invalid escape sequence '\/'
4:40: W605 invalid escape sequence '\S'
4:44: W605 invalid escape sequence '\S'
4:50: W605 invalid escape sequence '\/'
4:53: W605 invalid escape sequence '\/'
4:55: W605 invalid escape sequence '\S'
24:80: E501 line too long (81 > 79 characters)


**Ответ на задание 6:**

In [13]:
print('{:<25s}{:>5s}'.format('word', 'count'))
for word, count in words.most_common(20):
    print('{:<25s}{:5>d}'.format(word, count))

word                     count
the                      133
to                       96
a                        74
and                      68
i                        68
you                      54
of                       51
is                       50
it                       49
for                      43
in                       41
that                     35
me                       31
on                       30
my                       26
be                       25
its                      24
this                     22
are                      22
your                     21


# Задание 7

Здесь я просто складываю пользователей из неудалённых твитов в *users_followers* - Counter, если они встречаются в первый раз. Ключами словаря сделал кортежи из id пользователя и имени, показалось удобнее.

In [14]:
users_followers = Counter()

for tweet in tweets:
    if list(tweet.keys())[0] != 'delete':
        id = tweet['user']['id']
        name = tweet['user']['name']
        followers_count = tweet['user']['followers_count']

        if (id, name) not in users_followers:
            users_followers[(id, name)] = followers_count

**Ответ на задание 7:** *(снова криво из-за не ascii символов)*

In [15]:
print('{:<25s}{:>30s}'.format('author', 'number of followers'))
for user, count in users_followers.most_common(10):
    print('{:<25s}{:>30d}'.format(user[1], count))

author                              number of followers
Filosofía♕                                      2521403
FITNESS Magazine                                1491309
malaysiakini.com                                1206759
NYT Science                                     1137374
Gramática                                        625463
TGRT Haber                                       392472
The Sun Football ⚽                               383698
Melbourne, Australia                             374222
Roznama Express                                  318189
💞 ცųཞɠɛཞცơơɠıɛ 💞                                 311319


# Задание 8

Заведя каунтер, мы можем просто для каждого не удалённого твита доставать source, применять регулярку, доставая нужный кусочек строки, и вносить это в каунтер. Основную сложность в задании представляет регулярка. Я попытался разбить её по строчкам, и описать, как когда-то делал раньше, но в такой записи она не работала почему-то :( И с плюсом между `r'кусочками'` пробовал, и без. Оставил закомментированную, все равно может помочь. Арабский шрифт выводится неверно, не смог поправить.

In [16]:
def get_source(line):
    #     re_source = re.compile(
    #         r'<a href=\"[hH]ttp[s]?:\/\/' # http(s):// (в тексте встречается Http...)
    #         r'\S+.\S+'                    # site.domain
    #         r'(?:\/|\/\S+)*\"'            # необязательное / или /folder
    #                                       # повторённое несколько раз
    #         r'rel=\"\S+\">'
    #         r'((?:[^ \n]+|[^ \n]+ | )+)'  # одно слово либо слово + пробел, либо пробел
    #                                       # потом заканчиваем или повторяем снова
    #                                       # так получается любое количество слов
    #         r'<\/a>')
    re_source = re.compile(
        r'<a href=\"[hH]ttp[s]?:\/\/\S+.\S+(?:\/|\/\S+)*\" rel=\"\S+\">((?:[^ \n]|[^ \n]+ | )+)<\/a>')
    clear_line = re_source.match(line)
    return clear_line.group(1)


sources = Counter()
for tweet in tweets:
    if list(tweet.keys())[0] != 'delete':
        sources[get_source(tweet['source'])] += 1

3:80: E501 line too long (87 > 79 characters)
8:80: E501 line too long (89 > 79 characters)
9:80: E501 line too long (83 > 79 characters)
10:80: E501 line too long (82 > 79 characters)
13:80: E501 line too long (102 > 79 characters)


**Ответ на задание 8:**

In [17]:
print('{:<40s}{:>30s}'.format('platform', 'count'))
for platform, count in sources.most_common(10):
    print('{:<40s}{:>30d}'.format(platform, count))

platform                                                         count
Twitter for iPhone                                                 800
Twitter for Android                                                695
Twitter Web Client                                                 140
twittbot.net                                                       122
Twitter Lite                                                        51
Twitter for iPad                                                    28
TweetDeck                                                           23
Facebook                                                            17
IFTTT                                                               14
تطبيق قرآني                                                         10
